# Задание1

Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

* если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
* для источников paid и email из России ставим ad;
* для источников paid и email не из России ставим other;
* все остальные варианты берём из traffic_source без изменений.

In [ ]:
import pandas as pd

log = pd.read_csv(r'dop\visit_log.csv', sep=';')

#если источник traffic_source равен Yandex или Google, то в source_type ставится organic
log.loc[log.traffic_source.str.contains('yandex|google'),'source_type'] = 'organic'

#для источников paid и email из России ставим ad
log.loc[(log.traffic_source.str.contains('paid|email')) & (log.region == 'Russia'),'source_type'] = 'ad'

#для источников paid и email не из России ставим other
log.loc[(log.traffic_source.str.contains('paid|email')) & (log.region != 'Russia'),'source_type'] = 'other'

log.loc[pd.isna(log.source_type),'source_type'] = log['traffic_source']

# Задание 2 

В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

* Прочитайте содержимое файла с датафрейм.
* Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.


In [ ]:
import pandas as pd

urls = pd.read_table(r'dop\URLs.txt')
urls = urls[urls.url.str.contains(r'/\d{8}-')]

# Задание 3 

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [ ]:
import pandas as pd

ratings = pd.read_csv(r"ml-latest-small\ratings.csv")
ratings.head()

def get_timelife(timestamp):
    min = timestamp.min()
    max = timestamp.max()
    
    return (max - min);
    
timelife = ratings.groupby('userId').agg({'timestamp' : [get_timelife, 'count']})

timelife[timelife['timestamp']['count'] > 100].timestamp.get_timelife.mean()


# Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”» ---- Ноутбуки к лекции «Продвинутый pandas»).
Нужно сформировать две таблицы:

* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
* аналогичную таблицу по типам выручки с указанием адреса клиента.


Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

---

Дана статистика услуг перевозок клиентов компании по типам:

* rzd - железнодорожные перевозки
* auto - автомобильные перевозки
* air - воздушные перевозки
* client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [ ]:
import pandas as pd
from functools import reduce

revenue = reduce(lambda a, b: a.merge(b, how='outer', on='client_id'), [rzd, auto, air]).fillna(0)

revenue_with_client_base = revenue.merge(client_base, how='outer', on='client_id')

revenue_with_client_base

In [ ]:
# Способ без объединений

import pandas as pd

rzd.set_index('client_id', inplace=True)
auto.set_index('client_id', inplace=True)
air.set_index('client_id', inplace=True)

revenue = pd.concat([rzd, auto, air], axis=1).fillna(0)

revenue_with_client_base = revenue.merge(client_base, how='outer', on='client_id')

revenue_with_client_base